# Imports


In [1]:
import torchvision
from copy import deepcopy
from utils.transforms import *
from utils.dataset import *

# Data

In [ ]:
physionet = PhysionetDataset(train=True)
slp = SLPDataset(train=True)

# Baseline

- Basic normalization
  - normalize values between 0 and 1 because images from the different datasets have different value ranges in the images
  - equalizeHist to have a unified brightness and contrast
  - normalize mean because it normalizes statistical properties of the images and is a recommended and often used step
  - same for standardization
- warum diese reihenfolge?

In [ ]:
torchvision.transforms.Compose(
    [
        Resize((32, 64)),
        Standardize(),
        NormalizeValues(),
        EqualizeHist(),
    ]
),

# Agressive Thresholding

In [ ]:
torchvision.transforms.Compose(
    [
        Resize((32, 64)),
        Standardize(),
        NormalizeValues(),
        EqualizeHist(),
        Blur((3, 3)),
        Threshold(lambda img: np.median(img[img > 0.0])),
    ]
),

# Sobel

In [ ]:
torchvision.transforms.Compose(
    [
        Resize((32, 64)),
        Standardize(),
        NormalizeValues(),
        EqualizeHist(),
        Blur((3, 3)),
        Threshold(lambda img: np.median(img[img > 0.0])),
        Sobel(),
        Threshold(lambda img: np.median(img[img > 0.0])),
    ]
),

# Rolling Ball

In [ ]:
plot_samples(
    samples,
    ncols=10,
    transform=torchvision.transforms.Compose(
        [
            *normalizations,
            RollingBall(radius=5),
            Blur((3, 3)),
            Threshold(lambda img: np.median(img[img > 0.0])),
        ]
    ),
)

# Fourier

In [ ]:
plot_samples(
    samples,
    ncols=10,
    transform=torchvision.transforms.Compose(
        [
            *normalizations,
            Denoise(),
            Blur((3, 3)),
            HighPass(rad=50),
            Blur((3, 3)),
            Threshold(lambda img: np.median(img[img > 0.0])),
        ]
    ),
)

# Improving SLP

- hier zwei szenarien testen: einmal diese transformation auf beide datensätze anwenden und einmal nur auf slp und agressive thresholding oder so auf physionet
- interessante sache: später resizen behält in den schritten davor noch mehr details, womit besser preprocessed werden kann
- Potential: Menschen auf slp bildern vergrößern, damit die auf allen druckbildern ungefähr gleich groß sind

In [ ]:
torchvision.transforms.Compose(
    [
        Standardize(),
        NormalizeValues(),
        EqualizeHist(),
        Blur((3, 3)),
        Denoise(),
        Threshold(lambda img: np.median(img[img > 0.0])),
        Close((3, 3), ktype=cv2.MORPH_CROSS, iterations=3),
        Blur((3, 3)),
        Resize((32, 64)),
    ]
),

# ZCA

In [ ]:
physionet_zca = deepcopy(physionet)
slp_zca = deepcopy(slp)
physionet_zca.x = zca(physionet_zca.x, epsilon=0.1)
slp_zca.x = np.load("./data/SLP/data_zca.npy")